In [1]:
def get_target_distribution(df):
    print("-1: ", df[df['target']==-1].size)
    print(" 0: ", df[df['target']==0].size)
    print(" 1: ", df[df['target']==1].size)

In [ ]:
def get_training_timeseries(freq='D', end_training, context_length = 50, prediction_length=1, tickers)

    # we use 2 hour frequency for the time series
    #freq = freq

    # we predict for 7 days
    #prediction_length = prediction_length

    # we also use 7 days as context length, this is the number of state updates accomplished before making predictions
    #context_length = context_length

    end_training = pd.Timestamp(end_training, freq=freq)

    timeseries = []
    
    for ID,ticker in list(enumerate(tickers)):
        ticker = stock_data_preprocessed.loc[(slice(None), ticker), :]
        if ticker.index[0][0]<end_training:
            timeseries.append(ticker)
    return timeseries

In [ ]:
def write_json_dataset(filename, data): 
    with open(filename, 'wb') as f:
        # for each of our times series, there is one JSON line
        for d in data:
            json_line = json.dumps(d) + '\n'
            json_line = json_line.encode('utf-8')
            f.write(json_line) 

In [ ]:
def copy_to_s3(local_file, s3_path, override=False):
    assert s3_path.startswith('s3://')
    split = s3_path.split('/')
    bucket = split[2]
    path = '/'.join(split[3:])
    buk = s3.Bucket(bucket)
    
    if len(list(buk.objects.filter(Prefix=path))) > 0:
        if not override:
            print('File s3://{}/{} already exists.\nSet override to upload anyway.\n'.format(s3_bucket, s3_path))
            return
        else:
            print('Overwriting existing file')
    with open(local_file, 'rb') as data:
        print('Uploading file to {}'.format(s3_path))
        buk.put_object(Key=path, Body=data)

In [ ]:
def encode_request(instance, num_samples, quantiles):
        configuration = {
            "num_samples": num_samples,
            "output_types": ["quantiles"],
            "quantiles": quantiles
        }
        
        http_request_data = {
            "instances": [instance],
            "configuration": configuration
        }
        
        return json.dumps(http_request_data).encode('utf-8')


In [ ]:
def get_stock_prediction(ticker,date):
    try:
        date_pred = pd.Timestamp(date, freq='D')
        date_start = date_pred-50
        pred_df = stock_data_preprocessed.loc[(slice(str(date_start),str(date_pred)), ticker), :]
        result_df = pred_df.loc[(slice(str(date_pred),str(date_pred)), ticker), :]

        pred = {
                "start": str(date_pred),
                "target": pred_df['target'][date_start:date_pred-1].tolist(),
                "dynamic_feat": pred_df[['Adj Close','Volume']][date_start:date_pred].values.T.tolist()
        }

        req = encode_request(instance=pred, num_samples=50, quantiles=['0.1', '0.5', '0.9'])
        res = predictor.predict(req)

        prediction_data = json.loads(res.decode('utf-8'))
        pred = round(prediction_data['predictions'][0]['quantiles']['0.5'][0])
        result_df['prediction'] = pred
        return result_df
    except:
        print('{} did not trade today.'.format(ticker))

In [ ]:
def get_prediction_accuracy(ticker, date_range):
    ticker = str(ticker)
    i = 0
    target = []
    prediction = []

    for date in date_range:
        target.append(get_stock_prediction(ticker, date)['target'].values[0])
        prediction.append(int(get_stock_prediction(ticker, date)['prediction'].values[0]))
    target = list(np.array(target).reshape(252))
    prediction = list(np.array(prediction).reshape(252))
    data = {'target': list(target), 'prediction': list(prediction)}
    prediction_df = pd.DataFrame(data=data,index=date_index, columns=['target','prediction'])
    
    return accuracy_score(target, prediction), prediction_df